In [1]:
from __future__ import print_function

import tensorflow as tf

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

%config InlineBackend.figure_format='retina'
%matplotlib inline

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina', quality=100)

import warnings
warnings.filterwarnings('ignore')

2022-11-08 16:48:19.108675: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-08 16:48:19.292995: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-08 16:48:20.244936: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-08 16:48:20.245010: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
import hopsworks
project = hopsworks.login(
    host="2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai",
    project="electricity",
    api_key_value="xIGaDiwBZpXxLYRQ.SgheZwqHTZteASnO1PtPMC4HMgco2OVD6VK6xZN3sKCy96FQT27rF0JbBuFivTTA",
)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai:443/p/7287
Connected. Call `.close()` to terminate connection gracefully.


In [58]:
feature_view = fs.get_feature_view(
    name = 'se3_da_pricing_model',
    version = 5
)

In [66]:
X_train, y_train = feature_view.get_training_data(1)
train_data = pd.concat([X_train, y_train], axis=1)
train_data.sort_values(['time'], inplace=True)
train_data.drop(['time'], axis=1, inplace=True)

In [68]:
X_val, y_val = feature_view.get_training_data(2)
val_data = pd.concat([X_val, y_val], axis=1)
val_data.sort_values(['time'], inplace=True)
val_data.drop(['time'], axis=1, inplace=True)

In [69]:
val_data

,mean_temp_per_day_se3,mean_wind_speed_se3,precipitaton_amount_se3,total_sunshine_time_se3,mean_cloud_perc_se3,is_holiday,se4se3,se3se4,dk1se3,se3dk1,...,se3fi,no1se3,se3no1,fossil_gas,hydro_water_reservoir,nuclear,other,solar,wind_onshore,price
69,0.723404,0.294852,0.000000,0.456760,0.693923,1,1.0,0.531546,1.000000,0.372905,...,0.003311,1.0,0.746137,NaN,NaN,NaN,NaN,NaN,NaN,54.93
169,0.723404,0.294852,0.000000,0.456760,0.693923,1,1.0,0.558824,1.000000,0.138268,...,0.003311,1.0,0.647682,NaN,NaN,NaN,NaN,NaN,NaN,58.51
236,0.723404,0.294852,0.000000,0.456760,0.693923,1,1.0,0.523008,0.059432,0.000000,...,0.003311,1.0,0.600883,NaN,NaN,NaN,NaN,NaN,NaN,60.71
204,0.723404,0.294852,0.000000,0.456760,0.693923,1,1.0,0.546252,0.062016,0.000000,...,0.003311,1.0,0.536424,NaN,NaN,NaN,NaN,NaN,NaN,64.27
152,0.723404,0.294852,0.000000,0.456760,0.693923,1,1.0,0.558112,0.418605,0.000000,...,0.003311,1.0,0.475055,NaN,NaN,NaN,NaN,NaN,NaN,68.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,0.744681,0.228549,0.289855,0.004834,0.938785,1,1.0,0.889469,1.000000,0.997207,...,0.912252,1.0,0.241060,NaN,NaN,NaN,NaN,NaN,NaN,20.03
146,0.744681,0.228549,0.289855,0.004834,0.938785,1,1.0,0.912002,1.000000,0.997207,...,0.987583,1.0,0.192936,NaN,NaN,NaN,NaN,NaN,NaN,21.41
317,0.744681,0.228549,0.289855,0.004834,0.938785,1,1.0,0.966082,1.000000,0.969274,...,0.999172,1.0,0.089625,NaN,NaN,NaN,NaN,NaN,NaN,22.00
199,0.744681,0.228549,0.289855,0.004834,0.938785,1,1.0,0.975332,1.000000,0.530726,...,0.999172,1.0,0.143929,NaN,NaN,NaN,NaN,NaN,NaN,25.90


In [81]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
               df_train, df_val, label_columns=None, batch_size=32):
        # Store the raw data.
        self.df_train = df_train
        self.df_val = df_val
        
        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                               enumerate(df_train.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

        self.batch_size = batch_size
        
    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])

    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels

    #make_dataset method will take a time series DataFrame and convert it to a tf.data.Dataset of (input_window, label_window) 
    # pairs using the tf.keras.utils.timeseries_dataset_from_array function:
    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
          data=data,
          targets=None,
          sequence_length=self.total_window_size,
          sequence_stride=1,
          shuffle=False,
          batch_size=self.batch_size,)    
        ds = ds.map(self.split_window)
        ds = ds.repeat(1000)
        ds = ds.prefetch(10)  
        return ds

    @property
    def train(self):
        return self.make_dataset(self.df_train)
    
    @property
    def val(self):
        return self.make_dataset(self.df_val)

In [82]:
n_step_window = WindowGenerator(df_train=train_data, df_val = val_data, input_width=4, label_width=4, shift=1, label_columns=["price"])
n_step_window

Total window size: 5
Input indices: [0 1 2 3]
Label indices: [1 2 3 4]
Label column name(s): ['price']

In [83]:
for example_inputs, example_labels in n_step_window.train.take(1):
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')

Inputs shape (batch, time, features): (32, 4, 23)
Labels shape (batch, time, features): (32, 4, 1)


In [84]:
def build_model(input_dim):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv1D(filters = 64, kernel_size=1, padding='same', kernel_initializer="uniform", input_shape=(input_dim[0], input_dim[1])))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))        

    model.add(tf.keras.layers.Conv1D(filters = 32, kernel_size= 1,padding='same',  kernel_initializer="uniform"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))       

    model.add(tf.keras.layers.Conv1D(filters = 16, kernel_size= 1,padding='same',  kernel_initializer="uniform"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))   
    model.add(tf.keras.layers.MaxPooling1D(pool_size=1, padding='same'))       
    
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=16, return_sequences=True))) 
    model.add(tf.keras.layers.Dropout(rate=0.1))
    model.add(tf.keras.layers.Dense(units=4))
    
    model.summary()
    model.compile(loss='mae', optimizer='adam')
    return model

In [85]:
model = build_model([4, 23])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_12 (Conv1D)          (None, 4, 64)             1536      
                                                                 
 batch_normalization_12 (Bat  (None, 4, 64)            256       
 chNormalization)                                                
                                                                 
 leaky_re_lu_12 (LeakyReLU)  (None, 4, 64)             0         
                                                                 
 conv1d_13 (Conv1D)          (None, 4, 32)             2080      
                                                                 
 batch_normalization_13 (Bat  (None, 4, 32)            128       
 chNormalization)                                                
                                                                 
 leaky_re_lu_13 (LeakyReLU)  (None, 4, 32)            

In [108]:
from timeit import default_timer as timer
start = timer()

history = model.fit(n_step_window.train,
                    epochs=50,
                    verbose=0,
                    steps_per_epoch=200,
                    validation_data=n_step_window.val,
                    validation_steps=1,                    
                   )
end = timer()
print(end - start)

57.32263918800163


In [99]:
history_dict = history.history
history_dict['loss']

[92.507568359375,
 89.81761932373047,
 92.40052795410156,
 90.04080963134766,
 88.9478530883789,
 87.71501922607422,
 87.83935546875,
 89.05108642578125,
 89.62106323242188,
 89.92597198486328,
 91.89015197753906,
 91.1740951538086,
 91.62187194824219,
 91.42916870117188,
 91.35385131835938,
 90.86158752441406,
 91.07252502441406,
 89.548095703125,
 91.16883850097656,
 90.62126159667969,
 90.98314666748047,
 91.0089111328125,
 90.9409408569336,
 90.99951934814453,
 90.9967041015625,
 90.86024475097656,
 91.07238006591797,
 90.50086212158203,
 90.72258758544922,
 90.57144927978516,
 91.43778228759766,
 91.5215835571289,
 90.70804595947266,
 91.37606811523438,
 90.09455871582031,
 90.76985931396484,
 90.77175903320312,
 90.41107940673828,
 89.72518157958984,
 90.39105224609375,
 90.85832214355469,
 91.12159729003906,
 90.94121551513672,
 90.86598205566406,
 90.59734344482422,
 90.52544403076172,
 90.70274353027344,
 90.6782455444336,
 90.66381072998047,
 90.07837677001953,
 90.8280410766

In [107]:
history_dict['val_loss'][99]

32.502159118652344

In [90]:
export_path = "electricity_price_model"
print('Exporting trained model to: {}'.format(export_path))
tf.saved_model.save(model, export_path) 

Exporting trained model to: electricity_price_model
2022-11-08 17:53:08,233 WARNING: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, lstm_cell_13_layer_call_fn, lstm_cell_13_layer_call_and_return_conditional_losses while saving (showing 5 of 7). These functions will not be directly callable after loading.
INFO:tensorflow:Assets written to: electricity_price_model/assets
2022-11-08 17:53:10,013 INFO: Assets written to: electricity_price_model/assets


In [91]:
mr = project.get_model_registry()
metrics={'loss': history_dict['val_loss'][0]} 

mr_model = mr.tensorflow.create_model(
    name="electricity_price_prediction_model",
    metrics=metrics,
    description="Daily electricity price prediction model."
)

Connected. Call `.close()` to terminate connection gracefully.
